데이터 분석

In [2]:
import json

data_path = '/content/drive/MyDrive/롯데정보통신ai/1_인텐트_분류.json'

with open(data_path,'r') as f:
    json_data = json.load(f)

data_type = []
for data in json_data:
    data_type.append(data['인텐트'])

# label 유형
print(set(data_type)) # {'결제_일반_질문', '결제_할인_질문', '구매_예약_요청', '배송_날짜_요청', '제품_시용_질문', '제품_정보_질문'}

for i in list(set(data_type)): # i[0] = 90, i[1] = 133, i[2] = 154, i[3] = 78, i[4] = 68, i[5] = 110
    print(f'{i}의 데이터 갯수 {data_type.count(i)}')

# data 개수
len(json_data) # 633


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/롯데정보통신ai/1_인텐트_분류.json'

In [3]:
data_type = []
for data in json_data:
    data_type.append(data['발화자'])

# label 유형
print(set(data_type)) # {'결제_일반_질문', '결제_할인_질문', '구매_예약_요청', '배송_날짜_요청', '제품_시용_질문', '제품_정보_질문'}

for i in list(set(data_type)): # i[0] = 90, i[1] = 133, i[2] = 154, i[3] = 78, i[4] = 68, i[5] = 110
    print(f'{i}의 데이터 갯수 {data_type.count(i)}')

# data 개수
len(json_data) # 633

NameError: name 'json_data' is not defined

중복 제거

In [ ]:
import pandas as pd

json2pd = pd.DataFrame(json_data)
print(json2pd['발화문'].nunique())
print(json2pd.drop_duplicates(subset='발화문'))

bert를 이용한 classification model

In [4]:
!pip install transformers


feature가 2개니까 multi-task-learning으로 가자잇
1. 소 의도분류
2. 대 의도분류
3. user 구분

json-> tokenizer하기 위한 전처리

In [5]:
import random

total_data = []

# train # 9
# test # 1

for data in json_data:
    temp = []
    temp.append([data['발화자']][0])
    # if data['발화자'] == 'c':
        # temp.append(10)
    # if data['발화자'] == 's':
        # temp.append(11)
    temp.append([data['발화문']][0])
    if data['인텐트'] == '결제_일반_질문':
        temp.append(0)
    if data['인텐트'] == '결제_할인_질문':
        temp.append(1)
    if data['인텐트'] == '구매_예약_요청':
        temp.append(2)
    if data['인텐트'] == '배송_날짜_요청':
        temp.append(3)
    if data['인텐트'] == '제품_시용_질문':
        temp.append(4)
    if data['인텐트'] == '제품_정보_질문':
        temp.append(5)
    total_data.append(temp)

NameError: name 'json_data' is not defined

In [ ]:
len(total_data)
print(total_data)

In [ ]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(total_data, test_size = 0.1, shuffle = True, random_state = 42)

In [ ]:
print(train_data)
print(test_data)

tokenizer

In [ ]:
!pip install konlpy

In [ ]:
from konlpy.tag import Okt
okt = Okt()
print(okt.morphs('네, 삼성건조기 설치하러 11월 1일에 방문할 예정입니다.',stem = True))
a = okt.morphs('네, 삼성건조기 설치하러 11월 1일에 방문할 예정입니다.',stem = True)

In [ ]:
# model = BertForSequenceClassification.from_pretrained('klue/bert-base', num_labels=2)

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification

model_name = 'klue/bert-base'
tokenizer = BertTokenizer.from_pretrained(model_name)
print(tokenizer('네, 삼성건조기 설치하러 11월 1일에 방문할 예정입니다.', truncation=True, padding =True))
print(tokenizer.tokenize('네, 삼성건조기 설치하러 11월 1일에 방문할 예정입니다.', truncation=True, padding =True))

In [ ]:
# train_tokenize = []
# for i,j,k in train_data:
#     train_tokenize.append(tokenizer(j,truncation = True, padding = True))
#     # print(i,j,k)

# print(train_tokenize)

In [ ]:
def convert_data2feature(datas, max_length, tokenizer):
    input_ids_features, attention_mask_features, token_type_ids_features, label_id_features = [], [], [], []

    count = 0
    for use,utt,lab in datas:
        count+= 1
        print(utt)
        # klue tokenize로 치환
        input_ids = tokenizer(utt)['input_ids'][:max_length]
        # print(input_ids)
        # print(input_ids[-1])
        # print([tokenizer.sep_token])
        input_ids[-1] =  3
        token_type_ids = tokenizer(utt)['token_type_ids'][:max_length]
        attention_masks = tokenizer(utt)['attention_mask'][:max_length]

        # padding 생성

        padding = [0] * (max_length - len(input_ids))
        input_ids += padding
        token_type_ids += padding
        attention_masks += padding
        assert max_length == len(input_ids) == len(token_type_ids) == len(attention_masks)
        label_id = lab

        # 변환한 데이터를 각 리스트에 저장
        input_ids_features.append(input_ids)
        attention_mask_features.append(token_type_ids)
        token_type_ids_features.append(attention_masks)
        label_id_features.append(label_id)
        # print(input_ids_features)

    # 변환한 데이터를 Tensor 객체에 담아 반환
    # print(input_ids_features)
    input_ids_features = torch.tensor(input_ids_features, dtype=torch.long)
    attention_mask_features = torch.tensor(attention_mask_features, dtype=torch.long)
    token_type_ids_features = torch.tensor(token_type_ids_features, dtype=torch.long)
    label_id_features = torch.tensor(label_id_features, dtype=torch.long)

    return input_ids_features, attention_mask_features, token_type_ids_features, label_id_features

In [ ]:
import torch


convert_data2feature(train_data, 64, tokenizer)
print(len(train_data))

In [ ]:
import torch.nn as nn
from transformers import BertPreTrainedModel, BertModel

class IntentClassifier(BertPreTrainedModel):

    def __init__(self, config):
        super(IntentClassifier, self).__init__(config)

        # BERT 모델
        self.bert = BertModel(config)

        # 히든 사이즈
        self.hidden_size = config.hidden_size

        # 분류할 라벨의 개수
        self.num_labels = config.num_labels


        # self.linear = nn.Linear(in_features=self.hidden_size, out_features=self.num_labels)
        self.fc1 = nn.Linear(in_features=self.hidden_size, out_features = 32)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(in_features=32, out_features=self.num_labels)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input_ids, attention_mask, token_type_ids):
        # BERT 출력에서 CLS에 대응하는 벡터 표현 추출
        # 선형 함수를 사용하여 예측 확률 분포로 변환

        outputs = self.bert(input_ids=input_ids, attention_mask = attention_mask, token_type_ids = token_type_ids)

        # (batch_size, max_length, hidden_size)
        bert_output = outputs[0] # CLS의 output

        # (batch_size, hidden_size)
        cls_vector = bert_output[:,0,:]

        # class_output : (batch_size, num_labels)
        cls_output = self.fc1(cls_vector)
        cls_output = self.relu(cls_output)

        cls_output = self.fc2(cls_output)
        # cls_output = self.softmax(cls_output)

        return cls_output

In [ ]:
from torch.utils.data import (DataLoader, TensorDataset, RandomSampler)
from transformers import BertConfig
import torch.optim as optim
import torch
import numpy as np

def train(config):
    train_input_ids_features, train_attention_mask_features, train_token_type_ids_features, train_label_id_features = convert_data2feature(train_data, 64, tokenizer)
    train_features = TensorDataset(train_input_ids_features, train_attention_mask_features, train_token_type_ids_features, train_label_id_features)
    train_dataloader = DataLoader(train_features, sampler=RandomSampler(train_features), batch_size = config['batch_size'])

    # test_input_ids_features, test_attention_mask_features, test_token_type_ids_features, test_label_id_features = convert_data2feature(test_data, 64, tokenizer)
    # test_features = TensorDataset(test_input_ids_features, test_attention_mask_features, test_token_type_ids_features, test_label_id_features)
    # test_dataloader = DataLoader(test_features, sampler=RandomSampler(test_features), batch_size = config['batch_size'])

    bert_config = BertConfig.from_pretrained(pretrained_model_name_or_path='klue/bert-base')
    setattr(bert_config, "num_labels", config["num_labels"])

    # model = IntentClassifier.from_pretrained(pretrained_model_name_or_path='klue/bert-base', config=bert_config).cuda()
    model = IntentClassifier.from_pretrained(pretrained_model_name_or_path='klue/bert-base', config=bert_config)
    # loss를 계산하기 위한 함수
    loss_func = nn.CrossEntropyLoss()

    # 모델 학습을 위한 optimizer
    optimizer = optim.Adam(model.parameters(), lr=2e-5)

    # 모델의 정확도를 저장하기 위한 변수
    max_accuracy = 0
    global_step = 0
    for epoch in range(config["epoch"]):
        model.train()

        total_loss = []
        for step, batch in enumerate(train_dataloader):
            # batch = tuple(t.cuda() for t in batch)
            batch = tuple(t for t in batch)
            input_ids, attention_mask, token_type_ids, label_id = batch

            # 역전파 단계를 실행하기 전에 변화도를 0으로 변경
            optimizer.zero_grad()

            # 모델 예측 결과
            hypothesis = model(input_ids, attention_mask, token_type_ids)

            # loss 계산
            loss = loss_func(hypothesis, label_id)

            total_loss.append(loss.data.item())
            # loss 값으로부터 모델 내부 각 매개변수에 대하여 gradient 계산
            loss.backward()
            # 모델 내부 각 매개변수 가중치 갱신
            optimizer.step()

            # batch 당 저장할게
            print("Average loss : {}".format(np.mean(total_loss)))

        # epoch 당 저장할게
        bert_config.save_pretrained(save_directory=config["output_dir_path"])
        model.save_pretrained(save_directory=config["output_dir_path"])

In [ ]:
def test(config):

    from transformers import BertTokenizer, BertForSequenceClassification

    model_name = 'klue/bert-base'
    tokenizer = BertTokenizer.from_pretrained(model_name)

    test_input_ids_features, test_attention_mask_features, test_token_type_ids_features, test_label_id_features = convert_data2feature(test_data, 64, tokenizer)
    test_features = TensorDataset(test_input_ids_features, test_attention_mask_features, test_token_type_ids_features, test_label_id_features)
    test_dataloader = DataLoader(test_features, sampler=RandomSampler(test_features), batch_size = config['batch_size'])

    # train_input_ids_features, train_attention_mask_features, train_token_type_ids_features, train_label_id_features = convert_data2feature(train_data, 64, tokenizer)
    # train_features = TensorDataset(train_input_ids_features, train_attention_mask_features, train_token_type_ids_features, train_label_id_features)
    # train_dataloader = DataLoader(train_features, sampler=RandomSampler(train_features), batch_size = config['batch_size'])



    bert_config = BertConfig.from_pretrained(pretrained_model_name_or_path=config['output_dir_path'])
    model = IntentClassifier.from_pretrained(pretrained_model_name_or_path='/content/drive/MyDrive/롯데정보통신ai/model.safetensors', config=bert_config).cuda()

    model.eval()

    total_hypothesis, total_labels = [], []
    for idx, batch in enumerate(test_dataloader):
        batch = tuple(t.cuda() for t in batch)
        input_ids, attention_mask, token_type_ids, label_id = batch

        with torch.no_grad():
            # 모델 예측 결과
            # hypothesis : (batch_size, num_labels)
            hypothesis = model(input_ids, attention_mask, token_type_ids)
            # 모델의 출력값에 softmax와 argmax 함수를 적용
            hypothesis = torch.argmax(hypothesis, dim=-1)

        # Tensor를 리스트로 변경
        hypothesis = hypothesis.cpu().detach().numpy().tolist()
        label_id = label_id.cpu().detach().numpy().tolist()

        total_hypothesis.extend(hypothesis)
        total_labels.extend(label_id)

        for index in range(len(input_ids)):
            input_tokens = tokenizer.convert_ids_to_tokens(input_ids[index])
            print(input_tokens)
            input_sequence = tokenizer.convert_tokens_to_string(input_tokens[1:input_tokens.index(tokenizer.sep_token)])
            print(input_tokens)
            print(input_sequence)

            predict = [hypothesis[index]]
            correct = [label_id[index]]

            # if hypothesis[index] == 0:
                # predict = '결제_일반_질문'

            print("입력 : {}".format(input_sequence))
            print("출력 : {}, 정답 : {}\n".format(predict, correct))

        from sklearn.metrics import accuracy_score
        print("Accuracy= {0:f}\n".format(accuracy_score(total_hypothesis, total_labels)))

In [ ]:
config = {"mode": "train",
            # "train_data_path": os.path.join(root_dir, "train_datas_wordpiece.txt"),
            # "test_data_path": os.path.join(root_dir, "test_datas_wordpiece.txt"),
            "output_dir_path":'/content/drive/MyDrive/롯데정보통신ai',
            # "cache_dir_path": cache_dir,
            # "pretrained_model_name_or_path": "monologg/kobert",
            # "label_vocab_data_path": os.path.join(root_dir, "label_vocab.txt"),
            "num_labels": 6,
            "max_length": 64,
            "epoch":1,
            "batch_size":64,
            }

if (config["mode"] == "train"):
    train(config)
else:
    test(config)

In [ ]:
config = {"mode": "test",
            # "train_data_path": os.path.join(root_dir, "train_datas_wordpiece.txt"),
            # "test_data_path": os.path.join(root_dir, "test_datas_wordpiece.txt"),
            "output_dir_path":'/content/drive/MyDrive/롯데정보통신ai',
            # "cache_dir_path": cache_dir,
            # "pretrained_model_name_or_path": "monologg/kobert",
            # "label_vocab_data_path": os.path.join(root_dir, "label_vocab.txt"),
            "num_labels": 6,
            "max_length": 64,
            "epoch":1,
            "batch_size":64,
            }

if (config["mode"] == "train"):
    train(config)
else:
    test(config)

In [ ]:
config = {"mode": "test",
            # "train_data_path": os.path.join(root_dir, "train_datas_wordpiece.txt"),
            # "test_data_path": os.path.join(root_dir, "test_datas_wordpiece.txt"),
            "output_dir_path":'/content/drive/MyDrive/롯데정보통신ai',
            # "cache_dir_path": cache_dir,
            # "pretrained_model_name_or_path": "monologg/kobert",
            # "label_vocab_data_path": os.path.join(root_dir, "label_vocab.txt"),
            "num_labels": 6,
            "max_length": 64,
            "epoch":1,
            "batch_size":64,
            }

if (config["mode"] == "train"):
    train(config)
else:
    test(config)